In [73]:
import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI
import pyodbc
import os

load_dotenv('.env')

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
driver = os.getenv('DRIVER')

connection_string = f"DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=Yes"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [74]:
query = '''
        SELECT
            e.Descricao AS descricaoEquipamento,
            ie.Descricao AS descricaoInstalacaoEletrica,
            gc.Descricao AS GrupoDeCalculo,
            Calculo.Codigo AS CodigoCalculo,
            Calculo.Descricao,
            MAX(CAST(ExecucaoCalculoResultado.UltimaAtualizacao AS DATE)) AS UltimaAtualizacaoCalculo,
            ExecucaoCalculoResultado.Resultado AS HealtIndex
        FROM EngineCalculo.CalculoResultado As ExecucaoCalculoResultado
        INNER JOIN EngineCalculo.CalculoResultadoEquipamento 
            ON CalculoResultadoEquipamento.CalculoId = ExecucaoCalculoResultado.CalculoId 
            AND CalculoResultadoEquipamento.JobId = ExecucaoCalculoResultado.JobId
        INNER JOIN Treetech.ViewEngine_UltimoJob
            ON ViewEngine_UltimoJob.EquipamentoId = CalculoResultadoEquipamento.EquipamentoId
        INNER JOIN EngineCalculo.Calculo
            ON Calculo.Id = ExecucaoCalculoResultado.CalculoId
        INNER JOIN EngineCalculo.CalculoResultadoVariavel As EntradaVariavel 
            ON ExecucaoCalculoResultado.JobId = EntradaVariavel.JobId
        INNER JOIN EngineCalculo.RevisaoVariavel As RevisaoVariavel 
            ON RevisaoVariavel.VariavelId = EntradaVariavel.VariavelId 
            AND RevisaoVariavel.Revisao = EntradaVariavel.Revisao
        INNER JOIN [EngineCalculo].[Variavel] 
            ON Variavel.Id = EntradaVariavel.VariavelId 
        INNER JOIN Equipamento AS e 
            ON e.Id = CalculoResultadoEquipamento.EquipamentoId
        INNER JOIN EquipamentoInstalacaoEletrica AS eie 
            ON e.Id = eie.EquipamentoId
        INNER JOIN InstalacaoEletrica AS ie 
            ON  ie.Id = eie.InstalacaoEletricaId
        INNER JOIN Familia AS f 
            ON f.Id = e.FamiliaId
        INNER JOIN EngineCalculo.CategoriaVariavel AS cv 
            ON cv.Id = Variavel.CategoriaVariavelId
        LEFT JOIN EngineCalculo.GrupoCalculo AS gc 
            ON gc.Id = Calculo.GrupoCalculoId
        WHERE 
            Calculo.Descricao IN (
                'Índice de Estado do Equipamento', -- HI do Equipamento
                'Parte Ativa', -- HI do Subsistema
                'Comutador Sob Carga', -- HI do Subsistema
                'Acessórios', -- HI do Subsistema
                'Ambiente', -- HI do Subsistema
                'Tanque', -- HI do Subsistema
                'Resfriamento', -- HI do Subsistema
                'Bucha', -- HI do Subsistema
                'Preservação do Óleo Isolante' -- HI do Subsistema
            )
            AND e.Id = ?
        GROUP BY
            e.Descricao,
            ie.Descricao,
            gc.Descricao,
            Calculo.Codigo,
            Calculo.Descricao,
            ExecucaoCalculoResultado.Resultado
        ORDER BY MAX(CAST(ExecucaoCalculoResultado.UltimaAtualizacao AS DATE)) DESC
            , ie.Descricao
            , e.Descricao
'''


resultado = cursor.execute(query)
registro = resultado.fetchall()


Error: ('07002', '[07002] [Microsoft][ODBC Driver 18 for SQL Server]COUNT field incorrect or syntax error (0) (SQLExecDirectW)')

In [72]:
colunas = [column[0] for column in cursor.description]
data = [dict(zip(colunas, row)) for row in registro]
df = pd.DataFrame(data)
df


TypeError: 'NoneType' object is not iterable

In [65]:
colunas = [column[0] for column in cursor.description]
data = [dict(zip(colunas, row)) for row in registro]
df = pd.DataFrame(data)
df

,EquipamentoId,PJ,Subestacao,Familia,TUC,FaseEletrica,CodigoOperacional,EquipamentoDescricao,JobId,BCH_OLEO_TERC,...,IE_TR_BCH,BCH_CF_SEC,IE_TR_RESF_EV,INC_ATV_TMP,IE_TR_ACES_CAD,MON_TMP,BCH_TD_SEC,IE_TR_ATV_DGA,DIM_FINANCEIRO,REC_MOL_UMID_OLTC
0,1,ATE III,IT - ATE III,Autotransformador,570,A,INAT7-01,Autotransformador INAT7-01 Fase A,0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.03750,2.0
1,2,ATE III,IT - ATE III,Autotransformador,570,B,INAT7-01,Autotransformador INAT7-01 Fase B,0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.09375,2.0
2,3,ATE III,IT - ATE III,Autotransformador,570,C,INAT7-01,Autotransformador INAT7-01 Fase C,0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.09375,2.0
3,4,ATE III,IT - ATE III,Autotransformador,570,A,INAT7-02,Autotransformador INAT7-02 Fase A,0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.09375,2.0
4,5,ATE III,IT - ATE III,Autotransformador,570,B,INAT7-02,Autotransformador INAT7-02 Fase B,0,1.0,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.09375,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249,251,NTE,JNB III - TSN,Reator,330,C,"RE 66,6 JNB III - FVm 2","Reator RE 66,6 MVAr - 2 - FVm",0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.01250,0.0
250,252,NTE,JNB III - TSN,Reator,330,R,"RE 66,6 JNB III - FR 2-3","Reator RE 66,6 MVAr - 2-3 - FR",0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.01250,0.0
251,253,NTE,JNB III - TSN,Reator,330,A,"RE 66,6 JNB III - FBr 3","Reator RE 66,6 MVAr - 3 - FBr",0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.01250,0.0
252,254,NTE,JNB III - TSN,Reator,330,B,"RE 66,6 JNB III - FAz 3","Reator RE 66,6 MVAr - 3 - FAz",0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.01250,0.0


In [27]:
print(df.to_dict(orient='records'))

[{'Id': 1, 'Descricao': 'Autotransformador INAT7-01 Fase A', 'Nome': 'Autotransformador'}, {'Id': 2, 'Descricao': 'Autotransformador INAT7-01 Fase B', 'Nome': 'Autotransformador'}, {'Id': 3, 'Descricao': 'Autotransformador INAT7-01 Fase C', 'Nome': 'Autotransformador'}, {'Id': 4, 'Descricao': 'Autotransformador INAT7-02 Fase A', 'Nome': 'Autotransformador'}, {'Id': 5, 'Descricao': 'Autotransformador INAT7-02 Fase B', 'Nome': 'Autotransformador'}, {'Id': 6, 'Descricao': 'Autotransformador INAT7-02 Fase C', 'Nome': 'Autotransformador'}, {'Id': 7, 'Descricao': 'Autotransformador INAT7-03 Fase A', 'Nome': 'Autotransformador'}, {'Id': 8, 'Descricao': 'Autotransformador INAT7-03 Fase B', 'Nome': 'Autotransformador'}, {'Id': 9, 'Descricao': 'Autotransformador INAT7-03 Fase C', 'Nome': 'Autotransformador'}, {'Id': 10, 'Descricao': 'Autotransformador INAT7-01 Reserva', 'Nome': 'Autotransformador'}, {'Id': 11, 'Descricao': 'Autotransformador ATR-5 Fase A', 'Nome': 'Autotransformador'}, {'Id': 12

In [30]:
grouped_data = df.groupby('Nome')[['Descricao', 'Id']].apply(list).reset_index()

In [36]:

# Grouping and formatting the 'Descricao' column
grouped_data = df.groupby('Nome').apply(lambda group: group[['Descricao', 'Id']].to_dict(orient='records')).reset_index(name='Descricao')

# Creating the final result dictionary
result_dict = {
    "familyEquipment": [],
    "subsystems": [
        "Parte Ativa",
        "Comutador sobrecarga",
        "Parte Ativa",
        "Acessórios",
        "Ambiente",
        "Tanque",
        "Resfriamento",
        "Preservação do Óleo Isolante",
        "Bucha"
    ]
}

for idx, row in grouped_data.iterrows():
    equipment_data = {
        "Nome": row['Nome'],
        "Descricao": {x['Descricao']: x['Id'] for x in row['Descricao']}
    }
    result_dict["familyEquipment"].append(equipment_data)

print(result_dict)

{'familyEquipment': [{'Nome': 'Autotransformador', 'Descricao': {'Autotransformador INAT7-01 Fase A': 1, 'Autotransformador INAT7-01 Fase B': 2, 'Autotransformador INAT7-01 Fase C': 3, 'Autotransformador INAT7-02 Fase A': 4, 'Autotransformador INAT7-02 Fase B': 5, 'Autotransformador INAT7-02 Fase C': 6, 'Autotransformador INAT7-03 Fase A': 7, 'Autotransformador INAT7-03 Fase B': 8, 'Autotransformador INAT7-03 Fase C': 9, 'Autotransformador INAT7-01 Reserva': 10, 'Autotransformador ATR-5 Fase A': 11, 'Autotransformador ATR-5 Fase B': 12, 'Autotransformador ATR-5 Fase C': 13, 'Autotransformador ATR-5 Reserva': 14, 'Autotransformador 05T1 Fase A': 15, 'Autotransformador 05T1 Fase B': 16, 'Autotransformador 05T1 Fase C': 17, 'Autotransformador 05T2 Fase A': 18, 'Autotransformador 05T2 Fase B': 19, 'Autotransformador 05T2 Fase C': 20, 'Autotransformador 05T Reserva': 21, 'Autotransformador ATR-05T1 Fase A': 33, 'Autotransformador ATR-05T1 Fase B': 34, 'Autotransformador ATR-05T1 Fase C': 35